In [1]:
import numpy as np
import optuna

from image_classification.models.mlp import MLPClassifier
from image_classification.models.resnet import ResNetClassifier
from layers import ResNetBlock
import lightning.pytorch as pl
import lightning.pytorch.loggers as pl_loggers
import lightning.pytorch.callbacks as pl_callbacks

from util import set_seed
from weight_init import init_for_relu
from sklearn.model_selection import ParameterGrid
from datasets import FashionMNIST


In [ ]:
pl.seed_everything(42, True)

train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=16,
                                                           pin_memory=True,
                                                           num_workers=4,
                                                           persistent_workers=True)

n_classes = len(train_dl.dataset.classes)

# model = ResNetClassifier(
#     n_classes=n_classes,
#     opt='AdamW',
#     lr=1e-1,
#     wd=1e-4, ).apply(init_for_relu)

param_grid = {
    # 'layer_1': 2 ** np.arange(4, 6),
    'n_features': [(16, 32), (32, 64), (64, 128)],
    'lr': 10. ** -np.arange(1, 5),
    'wd': 10. ** -np.arange(1, 5),
}

for params in ParameterGrid(param_grid):
    model = MLPClassifier(
        input_sz=28 * 28,
        n_classes=n_classes,
        **params,
    ).apply(init_for_relu)

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name='fashion_mnist',
                                             sub_dir=model.__class__.__name__)

    trainer = pl.Trainer(max_epochs=5,
                         limit_train_batches=10,
                         limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )

    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )


In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    train_dl, valid_dl, test_dl = FashionMNIST.get_dataloaders(batch_size=16,
                                                               pin_memory=True,
                                                               num_workers=4,
                                                               persistent_workers=True)

    n_layers = trial.suggest_int("n_layers", 1, 3)
    n_features = [
        trial.suggest_int("n_features_l{}".format(i), 4, 128, log=True) for i in range(n_layers)
    ]
    params = {
        "n_features": n_features,
        "dropout": trial.suggest_float("dropout", 0.2, 0.5),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["AdamW", "RMSprop"]),
    }

    n_classes = len(train_dl.dataset.classes)
    model = MLPClassifier(
        input_sz=28 * 28,
        n_classes=n_classes,
        **params,
    ).apply(init_for_relu)

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name='fashion_mnist',
                                             sub_dir=model.__class__.__name__)

    trainer = pl.Trainer(max_epochs=5,
                         limit_train_batches=10,
                         limit_val_batches=10,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )

    trainer.fit(model=model,
                train_dataloaders=train_dl,
                val_dataloaders=valid_dl,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    return trainer.callback_metrics["accuracy/val"].item()


pruner = optuna.pruners.HyperbandPruner()

study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=600, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [ ]:
def objective(trial: optuna.trial.Trial) -> float:
    pl.seed_everything(42, True)

    params = {
        "dropout": trial.suggest_float("dropout", 0.1, 0.5, log=True),
        "lr": trial.suggest_float("lr", 1e-5, 1e-1, log=True),
        "wd": trial.suggest_float("wd", 1e-5, 1e-1, log=True),
        "opt": trial.suggest_categorical("opt", ["AdamW", "RMSprop"]),
    }

    data_module = FashionMNIST.FashionMNISTDataModule()
    n_classes = data_module.n_classes
    model = ResNetClassifier(
        n_classes=n_classes,
        **params).apply(init_for_relu)

    checkpoint_callback = pl.callbacks.ModelCheckpoint(
        save_top_k=1,
        monitor="accuracy/val",
        mode="max",
        filename="best-{epoch:02d}",
        save_last=True,
    )

    tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                             name=data_module.dataset_name,
                                             sub_dir=model.__class__.__name__)

    trainer = pl.Trainer(max_epochs=10,
                         limit_train_batches=200,
                         callbacks=[
                             checkpoint_callback,
                         ],
                         logger=tb_logger,
                         )

    trainer.fit(model,
                data_module,
                # ckpt_path='./results/fashion_mnist/version_5/checkpoints/best-epoch=04.ckpt'
                )

    return trainer.callback_metrics["accuracy/val"].item()


pruner = optuna.pruners.HyperbandPruner()
study = optuna.create_study(direction="maximize", pruner=pruner)
study.optimize(objective, n_trials=100, timeout=60 * 60, n_jobs=1)

print("Number of finished trials: {}".format(len(study.trials)))

print("Best trial:")
trial = study.best_trial

print("  Value: {}".format(trial.value))

print("  Params: ")
for key, value in trial.params.items():
    print("    {}: {}".format(key, value))

In [2]:
data_module = FashionMNIST.FashionMNISTDataModule()
ckpt_path = "results/FashionMNIST/version_2/checkpoints/last.ckpt"
model = ResNetClassifier.load_from_checkpoint(ckpt_path)

checkpoint_callback = pl.callbacks.ModelCheckpoint(
    save_top_k=1,
    monitor="accuracy/val",
    mode="max",
    filename="best-{epoch:02d}",
    save_last=True,
)

tb_logger = pl_loggers.TensorBoardLogger(save_dir='./results',
                                         name=data_module.dataset_name,
                                         sub_dir=model.__class__.__name__)

trainer = pl.Trainer(max_epochs=40,
                     callbacks=[
                         checkpoint_callback,
                     ],
                     logger=tb_logger,
                     )

trainer.fit(model,
            data_module,
            ckpt_path=ckpt_path,
            )

GPU available: True (cuda), used: True
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
Restoring states from the checkpoint path at results/FashionMNIST/version_2/checkpoints/last.ckpt
D:\anaconda3\envs\ailab\lib\site-packages\lightning\pytorch\callbacks\model_checkpoint.py:347: The dirpath has changed from './results\\FashionMNIST\\version_2\\checkpoints' to './results\\FashionMNIST\\version_12\\checkpoints', therefore `best_model_score`, `kth_best_model_path`, `kth_value`, `last_model_path` and `best_k_models` won't be reloaded. Only `best_model_path` will be reloaded.
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
Loading `train_dataloader` to estimate number of stepping batches.

  | Name       | Type       | Params
------------------------------------------
0 | classifier | Sequential | 1.2 M 
------------------------------------------
1.2 M     Trainable params
0         Non-trainable params
1.2 M     Total params
4.920 

Sanity Checking: |          | 0/? [00:00<?, ?it/s]

Training: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

...

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

Validation: |          | 0/? [00:00<?, ?it/s]

`Trainer.fit` stopped: `max_epochs=40` reached.
